# Attention Mechanism
---

# Imports & Device Setup

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


# Load Model & Tokenizer

In [ ]:
MODEL_PATH = "/kaggle/input/basic-model/saved_model/saved_gpt2_chatbot"

tokenizer = AutoModelForCausalLM.from_pretrained(MODEL_PATH)._tokenizer if False else AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    output_attentions=True
).to(device)

model.eval()


# Attention Inspection

In [ ]:
def inspect_attention(text):
    """
    Runs a forward pass through the model and visualizes attention 
    heatmap for Layer 1, Head 1.

    Args:
        text (str): Input sentence to inspect.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    attentions = outputs.attentions
    print(f"Number of layers: {len(attentions)}")
    print(f"Shape of first layer attention: {attentions[0].shape}")

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    attn_matrix = attentions[0][0, 0].cpu().numpy()

    plt.figure(figsize=(8, 6))
    sns.heatmap(attn_matrix, xticklabels=tokens, yticklabels=tokens, cmap="viridis")
    plt.title("Attention Heatmap - Layer 1, Head 1")
    plt.show()


In [ ]:
inspect_attention("Hi, how are you?")


# Generate Reply

In [ ]:
def generate_reply(history):
    """
    Generates a chatbot reply based on the entire conversation history.

    Args:
        history (str): Full conversation so far.

    Returns:
        str: Generated bot reply.
    """
    inputs = tokenizer(history, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=80,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.4,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


# Chatbot Loop

In [ ]:
def run_chatbot():
    """
    Runs an interactive multi-turn chatbot loop.
    Keeps adding messages to a conversation buffer and 
    generates context-aware responses.
    """
    print("\n=== Multi-Turn Chatbot ===\n")

    conversation_history = ""

    while True:
        user_msg = input("User: ").strip()
        if user_msg.lower() == "exit":
            break

        conversation_history += f"User: {user_msg}\nBot:"

        full_output = generate_reply(conversation_history)
        bot_reply = full_output.split("Bot:")[-1].strip()

        print(f"Bot: {bot_reply}")
        print("-" * 50)

        conversation_history += f" {bot_reply}\n"


In [ ]:
run_chatbot()


-----

# Continous Learning
---

# Imports & Model Loading

In [ ]:
import os
import json
import torch
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

MODEL_PATH = "/kaggle/input/basic-model/saved_model/saved_gpt2_chatbot"
DATA_PATH = "new_data.json"

# Load tokenizer & model
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATH)
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"Model running on: {device}")

# Chatbot Inference

In [ ]:
def chat_with_bot(user_input, max_length=150):
    """
    Generate a reply from the chatbot using GPT-2 model.
    """
    formatted_input = f"<user>{user_input}<bot>"
    input_ids = tokenizer.encode(formatted_input, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
        )

    decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    bot_reply = decoded_text.split("<bot>")[-1].strip()

    return bot_reply

# Data Saving

In [ ]:
def add_new_interaction(user_text, bot_text, feedback):
    """
    Save user-bot interaction + feedback to dataset file.
    """
    interaction = {
        "user": f"<user>{user_text}",
        "bot": f"<bot>{bot_text}",
        "feedback": feedback,
    }

    if not os.path.exists(DATA_PATH):
        with open(DATA_PATH, "w") as f:
            json.dump([], f)

    with open(DATA_PATH, "r+") as f:
        data = json.load(f)
        data.append(interaction)
        f.seek(0)
        json.dump(data, f, indent=2)

In [ ]:
def load_good_interactions(min_feedback=4):
    """
    Load all interactions that have feedback >= min_feedback.
    """
    if not os.path.exists(DATA_PATH):
        return []

    with open(DATA_PATH, "r") as f:
        data = json.load(f)

    return [
        f"{item['user']}{item['bot']}"
        for item in data
        if item["feedback"] >= min_feedback
    ]


In [ ]:
def create_training_file(pairs, file_path="temp_finetune.txt"):
    """
    Write selected pairs into a text file for GPT-2 fine-tuning.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for line in pairs:
            f.write(line + "\n")
    return file_path

# Fine-Tuning

In [ ]:
def fine_tune_on_new_data(train_file, output_dir="./updated_model"):
    """
    Fine-tune GPT-2 model on collected user-bot interactions.
    """
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        save_total_limit=1,
        logging_steps=20,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model(output_dir)

    print("Model updated successfully!")


In [ ]:
def check_and_train(threshold=200):
    """
    Check if enough high-quality feedback exists → retrain the model.
    """
    good_pairs = load_good_interactions()

    if len(good_pairs) >= threshold:
        print(f"Training triggered with {len(good_pairs)} high-quality examples!")

        train_file = create_training_file(good_pairs)
        fine_tune_on_new_data(train_file)

        # Clear file after successful training
        with open(DATA_PATH, "w") as f:
            json.dump([], f)

        print("Training complete. Data cleared!")
    else:
        print(
            f"Collected {len(good_pairs)} good examples. "
            f"{threshold - len(good_pairs)} more needed."
        )


# Chat Loop

In [ ]:
def run_chat(threshold=50):
    """
    Continuous chat loop + feedback collection + auto-training trigger.
    """
    print("Welcome! Type 'exit' to quit the chat.")

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() == "exit":
            print("Chat ended.")
            break

        # Generate bot response
        bot_reply = chat_with_bot(user_input)
        print("Bot:", bot_reply)

        # Collect feedback
        while True:
            try:
                feedback = int(input("Rate the bot reply (1–5): "))
                if 1 <= feedback <= 5:
                    break
                print("Rating must be between 1 and 5.")
            except:
                print("Please enter a valid number.")

        # Save interaction + check for training
        add_new_interaction(user_input, bot_reply, feedback)
        check_and_train(threshold=threshold)



In [ ]:
run_chat(threshold=50)